In [1]:
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup 
from selenium import webdriver

from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import requests
import re

In [2]:
#lets frst connect to the web driver
driver=webdriver.Chrome(r"chromedriver.exe")

In [3]:
url='https://www.amazon.in/'

In [4]:
driver.get(url)

In [5]:
user_input = input('Enter the product you want to search : ')

Enter the product you want to search : mouse


In [6]:
search_bar = driver.find_element_by_id("twotabsearchtextbox")    
search_bar.clear()                                               
search_bar.send_keys(user_input)                                   
search_button = driver.find_element_by_xpath('//div[@class="nav-search-submit nav-sprite"]/span/input') 
search_button.click()


In [7]:
start_page = 0
end_page = 3
urls = []
for page in range(start_page,end_page+1):
    try:
        page_urls = driver.find_elements_by_xpath('//a[@class="a-link-normal s-no-outline"]')
        
        for url in page_urls:
            url = url.get_attribute('href')     
            if url[0:4]=='http':                
                urls.append(url)                
        print("Product urls of page {} has been scraped.".format(page+1))
        
        nxt_button = driver.find_element_by_xpath('//li[@class="a-last"]/a')      
        if nxt_button.text == 'Next→':                                            
            nxt_button.click()                                                    
            time.sleep(5)                                                            
        elif driver.find_element_by_xpath('//li[@class="a-disabled a-last"]/a').text == 'Next→':    
            print("No new pages exist. Breaking the loop") 
            break
            
    except StaleElementReferenceException as e:             
        print("Stale Exception")
        next_page = nxt_button.get_attribute('href')        
        driver.get(next_page)

Product urls of page 1 has been scraped.
Product urls of page 2 has been scraped.
Product urls of page 3 has been scraped.
Product urls of page 4 has been scraped.


In [8]:
Brand_name=[]
Product_name=[]
Product_rating=[]
Product_NoOfRating=[]
Product_price=[]
Product_return=[]
Product_delivery=[]
Product_availability=[]
Product_details=[]
Product_url=[]

In [9]:
for url in urls[:4]:
    driver.get(url)                                                     
    print("Scraping URL = ", url)
    
    try:
        brand = driver.find_element_by_xpath('//a[@id="bylineInfo"]')      
        Brand_name.append(brand.text)
    except NoSuchElementException:
        Brand_name.append('-')
    
    try:
        name = driver.find_element_by_xpath('//h1[@id="title"]/span')      
        Product_name.append(name.text)
    except NoSuchElementException:
        Product_name.append('-')
    
    try:
        rating = driver.find_element_by_xpath('//span[@id="acrPopover"]')  
        Product_rating.append(rating.get_attribute("title"))
    except NoSuchElementException:
        Product_rating.append('-')
    
    try:
        n_rating = driver.find_element_by_xpath('//a[@id="acrCustomerReviewLink"]/span')     
        Product_NoOfRating.append(n_rating.text)
    except NoSuchElementException:
        Product_NoOfRating.append('-')
    
    try:
        price = driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]')  
        Product_price.append(price.text)
    except NoSuchElementException:
        Product_price.append('-')
    try:                                                                                     
        ret = driver.find_element_by_xpath('//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        Product_return.append(ret.text)
    except NoSuchElementException:
        Product_return.append('-')
    try:
        delivry = driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b')         
        Product_delivery.append(delivry.text)
    except NoSuchElementException:
        Product_delivery.append('-')
    
    try:
        avl = driver.find_element_by_xpath('//div[@id="availability"]/span')        
        Product_availability.append(avl.text)
    except NoSuchElementException:
        Product_availability.append('-')
    
    try:                                                                                    
        dtls = driver.find_element_by_xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]')
        Product_details.append('  ||  '.join(dtls.text.split('\n')))
    except NoSuchElementException:
        Product_details.append('-')
    
    Product_url.append(url)                                                            
    time.sleep(2)

Scraping URL =  https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A014876910SWTBTTV2M73&url=%2FLogitech-Silent-Wireless-Mouse-Charcoal%2Fdp%2FB01M72LILF%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dmouse%26qid%3D1622701757%26sr%3D8-1-spons%26psc%3D1&qualifier=1622701757&id=240697264827863&widgetName=sp_atf
Scraping URL =  https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A05258413JHWN62C3X8L2&url=%2FQuantum-QHM222-Mouse-Black%2Fdp%2FB00GN3TP4K%2Fref%3Dsr_1_2_sspa%3Fdchild%3D1%26keywords%3Dmouse%26qid%3D1622701757%26sr%3D8-2-spons%26psc%3D1&qualifier=1622701757&id=240697264827863&widgetName=sp_atf
Scraping URL =  https://www.amazon.in/Zebronics-Zeb-Transformer-M-Optical-Gaming-Effect/dp/B0819HZPXL/ref=sr_1_3?dchild=1&keywords=mouse&qid=1622701757&sr=8-3
Scraping URL =  https://www.amazon.in/Logitech-B170-Wireless-Mouse-Black/dp/B01J0XWYKQ/ref=sr_1_4?dchild=1&keywords=mouse&qid=1622701757&smid=

In [10]:

print(len(Brand_name),len(Product_name),len(Product_rating),len(Product_NoOfRating),len(Product_price),len(Product_delivery),len(Product_availability),len(Product_details),len(Product_url))

4 4 4 4 4 4 4 4 4


In [12]:
import pandas as pd
prdct=pd.DataFrame({})
prdct['Brand']=Brand_name
prdct['Name']=Product_name
prdct['Rating']=Product_rating
prdct['No.of rating']=Product_NoOfRating
prdct['Price']=Product_price
prdct['Delivery']=Product_delivery
prdct['Availability']=Product_availability
prdct['Details']=Product_details
prdct['URL']=Product_url

In [13]:
prdct

,Brand,Name,Rating,No.of rating,Price,Delivery,Availability,Details,URL
0,Visit the Logitech Store,"Logitech M221 Wireless Mouse, Silent Buttons, ...",4.5 out of 5 stars,"11,031 ratings",₹ 799.00,June 5 - 8,Available to ship in 1-2 days.,Ultra-quiet mouse with 90% reduced click sound...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Brand: QuantumHiTech Merchandising Pvt. Ltd.,Quantum QHM222 3-Button 1000DPI Wired Optical ...,3.8 out of 5 stars,"1,516 ratings",₹ 199.00,"Saturday, June 5",In stock.,"Left-handed or right, you can work comfortably...",https://www.amazon.in/gp/slredirect/picassoRed...
2,Visit the ZEBRONICS Store,Zebronics Zeb-Transformer-M Optical USB Gaming...,4.3 out of 5 stars,"14,757 ratings",₹ 442.00,"Saturday, June 5",In stock.,Zeb-Transformer-M is a Optical USB Gaming Mous...,https://www.amazon.in/Zebronics-Zeb-Transforme...
3,Visit the Logitech Store,"Logitech B170 Wireless Mouse, 2.4 GHz with USB...",4.4 out of 5 stars,"23,795 ratings",-,"Saturday, June 5",In stock.,Reliable Wireless Connection : Enjoy a wireles...,https://www.amazon.in/Logitech-B170-Wireless-M...


# 3

In [15]:
#lets frst connect to the web driver
driver=webdriver.Chrome(r"chromedriver.exe")


In [16]:
url='https://images.google.com/'

In [17]:
driver.get(url)

In [18]:
search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')    
search_bar.send_keys("fruits")       
search_button = driver.find_element_by_xpath('//*[@id="sbtc"]/button')    
search_button.click()

In [20]:
print("start scrolling to generate more images on the page...")
for _ in range(500):
    driver.execute_script("window.scrollBy(0,15000)")

start scrolling to generate more images on the page...


In [21]:
images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

In [22]:
img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)

152

In [23]:
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])

# 4

In [25]:
#lets frst connect to the web driver
driver=webdriver.Chrome(r"chromedriver.exe")

In [26]:
url='https://www.flipkart.com/'

In [27]:
driver.get(url)

In [28]:
prod = input(" Enter the name of the mobile phone : ")
time.sleep(3)

 Enter the name of the mobile phone : pixel4A


In [29]:
try:
    login_X_button = driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]')                 
    login_X_button.click()
except NoSuchElementException : 
    print("No Login page")
search_bar = driver.find_element_by_xpath('//*[@id="container"]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')    
search_bar.clear()               
search_bar.send_keys(prod)       
search_button = driver.find_element_by_xpath('//button[@class="L0Z3Pu"]')    
search_button.click()

No Login page


In [30]:
product_urls = []
urls = driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')
for url in urls:
    product_urls.append(url.get_attribute("href"))

In [31]:
Brand = []
Smartphone = []
Colour = []
RAM = []
Storage = []
PrimaryCamera = []
SecondaryCamera = []
DisplaySize = []
DisplayResolution = []
Processor = []
ProcessorCores = []
BatteryCapacity = []
BatteryType = []
Price = []
URL = []

In [32]:
for url in product_urls:
    driver.get(url)                                                       
    print("Scraping URL = ", url)
    URL.append(url)                                                          
    time.sleep(2)
    
    try:
        read_more = driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]')     
        read_more.click()
    except NoSuchElementException:
        print("Exception Occured. Moving to next page")
    
    try:
        brand = driver.find_element_by_xpath('//span[@class="B_NuCI"]')      
        Brand.append(brand.text.split()[0])
    except NoSuchElementException:
        Brand.append('-')
        
    try:
        price = driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')      
        Price.append(price.text)
    except NoSuchElementException:
        Price.append('-')
        
    try:
        name = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')      
        Smartphone.append(name.text)
    except NoSuchElementException:
        Smartphone.append('-')
    
    try:
        color = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      
        Colour.append(color.text)
    except NoSuchElementException:
        Colour.append('-')
    
    try:
        disp_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_chk.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  
        DisplaySize.append(disp_size.text)
    except NoSuchElementException:
        DisplaySize.append('-')
    
    try:
        disp_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_chk.text != "Display Features" : raise NoSuchElementException
        disp_res = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[2]/td[2]/ul/li')     
        DisplayResolution.append(disp_res.text)
    except NoSuchElementException:
        DisplayResolution.append('-')
    
    try:
        pro_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
        if pro_chk.text != "Processor Type" : raise NoSuchElementException
        processor = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')   
        Processor.append(processor.text)
    except NoSuchElementException:
        Processor.append('-')
    
    try:                                                                                     
        core_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[1]')
        if core_chk.text != "Processor Core" :
            core_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
            if core_chk.text != "Processor Core" : 
                raise NoSuchElementException
            else :
                cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')
        else :
            cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[2]/ul/li')
        ProcessorCores.append(cores.text)
    except NoSuchElementException:
        ProcessorCores.append('-')
    
    try:
        rom = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')         
        Storage.append(rom.text)
    except NoSuchElementException:
        Storage.append('-')
    
    try:
        ram = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                
        RAM.append(ram.text)
    except NoSuchElementException:
        RAM.append('-')
    
    try:                                                                                    
        pri_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        PrimaryCamera.append(pri_cam.text)
    except NoSuchElementException:
        PrimaryCamera.append('-')
    
    try:                                                                                    
        cam_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if cam_chk != "Secondary Camera" : 
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        SecondaryCamera.append(sec_cam.text)
    except NoSuchElementException:
        SecondaryCamera.append('-')
        
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
            bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')                
        BatteryCapacity.append(bat_cap.text)
    except NoSuchElementException:
        BatteryCapacity.append('-')
    
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr[2]/td[1]')
                if bat_chk.text != "Battery Type" : raise NoSuchElementException
                bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr[2]/td[2]/ul/li')
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr[2]/td[1]')
                if bat_chk.text != "Battery Type" : raise NoSuchElementException
                bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr[2]/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[1]')
            if bat_chk.text != "Battery Type" : raise NoSuchElementException
            bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[2]/ul/li')                
        BatteryType.append(bat_typ.text)
    except NoSuchElementException:
        BatteryType.append('-')

Scraping URL =  https://www.flipkart.com/google-pixel-4a-just-black-128-gb/p/itm023b9677aa45d?pid=MOBFUSBNAZGY7HQU&lid=LSTMOBFUSBNAZGY7HQUWHTF0C&marketplace=FLIPKART&q=pixel4A&store=tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=SEARCH&iid=a5a4f557-664d-430b-b63d-6c7a0423fe86.MOBFUSBNAZGY7HQU.SEARCH&ppt=hp&ppn=homepage&ssid=3trqk0lj800000001622702280140&qH=15245e820e90df42
Scraping URL =  https://www.flipkart.com/redmi-9-power-electric-green-64-gb/p/itmca7d78e222ed7?pid=MOBFYZ7AVAXXB2TH&lid=LSTMOBFYZ7AVAXXB2THVSNL0U&marketplace=FLIPKART&q=pixel4A&store=tyy%2F4io&srno=s_1_2&otracker=search&otracker1=search&fm=SEARCH&iid=a5a4f557-664d-430b-b63d-6c7a0423fe86.MOBFYZ7AVAXXB2TH.SEARCH&ppt=hp&ppn=homepage&ssid=3trqk0lj800000001622702280140&qH=15245e820e90df42
Scraping URL =  https://www.flipkart.com/redmi-9-power-mighty-black-64-gb/p/itm481ab234a6553?pid=MOBFYZ94UWPWQRNF&lid=LSTMOBFYZ94UWPWQRNFD8SF5T&marketplace=FLIPKART&q=pixel4A&store=tyy%2F4io&srno=s_1_3&otracker=search&otracker1

Scraping URL =  https://www.flipkart.com/redmi-9-power-electric-green-128-gb/p/itm6b72de6ad62b2?pid=MOBGYMD5HHNBHMCK&lid=LSTMOBGYMD5HHNBHMCKYPGDYX&marketplace=FLIPKART&q=pixel4A&store=tyy%2F4io&srno=s_1_23&otracker=search&otracker1=search&fm=SEARCH&iid=a5a4f557-664d-430b-b63d-6c7a0423fe86.MOBGYMD5HHNBHMCK.SEARCH&ppt=hp&ppn=homepage&ssid=3trqk0lj800000001622702280140&qH=15245e820e90df42
Scraping URL =  https://www.flipkart.com/samsung-galaxy-m21-iceberg-blue-64-gb/p/itm50475cfae8813?pid=MOBFXYA9ZMJG9VTY&lid=LSTMOBFXYA9ZMJG9VTYVS9HMZ&marketplace=FLIPKART&q=pixel4A&store=tyy%2F4io&srno=s_1_24&otracker=search&otracker1=search&fm=SEARCH&iid=a5a4f557-664d-430b-b63d-6c7a0423fe86.MOBFXYA9ZMJG9VTY.SEARCH&ppt=hp&ppn=homepage&ssid=3trqk0lj800000001622702280140&qH=15245e820e90df42


In [33]:
print(len(Brand), len(Smartphone), len(Processor), len(Price), len(URL))


24 24 24 24 24


In [34]:
import pandas as pd
prdct=pd.DataFrame({})
prdct["Brand"] = Brand
prdct["Smartphone"] =Smartphone
prdct["Colour"] =Colour
prdct["RAM"] =RAM
prdct["Storage(ROM)"] =Storage
prdct["Primary Camera"] =PrimaryCamera
prdct["Secondary Camera"] =SecondaryCamera 
prdct["Display Size"] =DisplaySize
prdct["Display Resolution"] =DisplayResolution 
prdct["Processor"] =Processor
prdct["Processor Cores"] =ProcessorCores 
prdct["Battery Capacity"] =BatteryCapacity
prdct["Battery Type"] =BatteryType
prdct["Price"] =Price
prdct["URL"] =URL

In [35]:
prdct

,Brand,Smartphone,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Battery Type,Price,URL
0,Google,Pixel 4a,Just Black,6 GB,128 GB,12.2MP Rear Camera,8MP Front Camera,14.76 cm (5.81 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 730G,Octa Core,3140 mAh,-,"₹29,999",https://www.flipkart.com/google-pixel-4a-just-...
1,REDMI,9 Power,Electric Green,4 GB,64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,lithium-polymer,"₹10,499",https://www.flipkart.com/redmi-9-power-electri...
2,REDMI,9 Power,Mighty Black,4 GB,64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,lithium-polymer,"₹10,499",https://www.flipkart.com/redmi-9-power-mighty-...
3,REDMI,9 Power,Fiery Red,4 GB,64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,lithium-polymer,"₹10,499",https://www.flipkart.com/redmi-9-power-fiery-r...
4,REDMI,Note 9,Aqua Green,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,-,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,Lithium Polymer,"₹10,999",https://www.flipkart.com/redmi-note-9-aqua-gre...
5,REDMI,9 Power,Blazing Blue,4 GB,64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,lithium-polymer,"₹10,499",https://www.flipkart.com/redmi-9-power-blazing...
6,REDMI,9 Power,Mighty Black,6 GB,128 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,Qualcomm® Snapdragon™ 662,Octa Core,6000 mAh,lithium-polymer,"₹12,999",https://www.flipkart.com/redmi-9-power-mighty-...
7,REDMI,Note 9,Arctic White,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,-,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,Lithium Polymer,"₹10,999",https://www.flipkart.com/redmi-note-9-arctic-w...
8,REDMI,Note 9,Shadow Black,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,-,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,Lithium Polymer,"₹10,999",https://www.flipkart.com/redmi-note-9-shadow-b...
9,Redmi,Redmi 9A,Nature Green,2 GB,32 GB,13MP Rear Camera,-,16.59 cm (6.53 inch),720 x 1600$$pixel,-,Octa Core,5000 mAh,-,"₹7,370",https://www.flipkart.com/redmi-9a-nature-green...


# 5

In [36]:
#lets frst connect to the web driver
driver=webdriver.Chrome(r"chromedriver.exe")

In [37]:
url='https://www.google.co.in/maps'

In [38]:
driver.get(url)

In [39]:
time.sleep(3)

In [40]:
city = input('Enter City Name : ')                                         
search = driver.find_element_by_id("searchboxinput")                       
search.clear()                                                             
time.sleep(3)
search.send_keys(city)                                                     
button = driver.find_element_by_id("searchbox-searchbutton")               
button.click()                                                             
time.sleep(3)

Enter City Name : dehradun


In [41]:
try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

URL Extracted:  https://www.google.co.in/maps/place/Dehradun,+Uttarakhand/@30.3253538,77.877046,11z/data=!3m1!4b1!4m5!3m4!1s0x390929c356c888af:0x4c3562c032518799!8m2!3d30.3164945!4d78.0321918
Latitude = 30.3253538, Longitude = 77.877046


# 6

In [51]:
#lets frst connect to the web driver
driver=webdriver.Chrome(r"chromedriver.exe")

In [52]:
url='https://trak.in/'

In [53]:
driver.get(url)

In [54]:
button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(button)

In [55]:
Date=[]
Startup=[]
Industry=[]
SubVertical=[]
Location=[]
Investor=[]
Investment=[]
Amount=[]

In [56]:
for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()

    dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        Date.append(d.text)

    sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        Startup.append(n.text)
    
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        Industry.append(n.text)
    
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        SubVertical.append(s.text)

    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        Location.append(l.text)
    
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        Investor.append(n.text)
    
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        Investment.append(n.text)
    
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        Amount.append(a.text)

In [57]:
fund=pd.DataFrame({})
fund['Date']=Date
fund['Startup']=Startup 
fund['Industry']=Industry
fund['SubVertical']=SubVertical
fund['Location']=Location
fund['Investor']=Investor
fund['Investment']=Investment
fund['Amount']=Amount

In [58]:
fund

,Date,Startup,Industry,SubVertical,Location,Investor,Investment,Amount
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


In [59]:

fund.to_csv("Indian Startups Company_2020.csv")

# 7

In [60]:
#lets frst connect to the web driver
driver=webdriver.Chrome(r"chromedriver.exe")

In [61]:
url='https://www.digit.in/'

In [62]:

driver.get(url)

In [63]:
name = []
price = []
OS = []
display = []
processor = []
weight = []
dimension = []
GPU = []

In [64]:
names=driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")
for i in names:
    name.append(i.text)
    
os=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[1]/div/div")
for i in os:
    OS.append(i.text)
    
displays=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[2]/div/div")
for i in displays:
    display.append(i.text)
    
processors=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[3]/div/div")
for i in processors:
    processor.append(i.text)
processor

weights=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")
weight_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")
for i in range(len(weights)):
        if weights[i].text=='Weight':
            weight.append(weight_spec[i].text)
        
dimension=[]
dims=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")
dims_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")
for i in range(len(dims)):
        if dims[i].text=='Dimension':
            dimension.append(dims_spec[i].text)

GPUs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")
GPUs_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")
for i in range(len(GPUs)):
        if GPUs[i].text=='Graphics Processor':
            GPU.append(GPUs_spec[i].text)
        
full_specs=[]
urls=driver.find_elements_by_xpath("//div[@class='full-specs']/span")
for i in urls:
    if i.get_attribute('data-href'):
        full_specs.append(i.get_attribute('data-href'))
    
for i in full_specs:
    driver.get(i)
    try:
        prices=driver.find_element_by_xpath("//div[@class='Block-price']/b")
        price.append(prices.text)
    except NoSuchElementException:
        price.append("No details available")

In [65]:
print(len(name),len(price),len(OS),len(display),len(processor),len(weight),len(dimension),len(GPU))


0 0 0 0 0 0 0 0


In [ ]:
prdct=pd.DataFrame({})
prdct['Name']=name
prdct['Price']=price
prdct['OS']=OS
prdct['Display']=display
prdct['Processor']=processor
prdct['Weight']=weight
prdct['Dimension']=dimension
prdct['GPU']=GPU
prdct